In [1]:
import cosmofast as cf
from astropy.io import fits
import numpy as np
ffE = fits.open('cosmofast/planck_18/data/legacy-simall/probEE')
ffB = fits.open('cosmofast/planck_18/data/legacy-simall/probBB')

In [2]:
ffB.info()

Filename: cosmofast/planck_18/data/legacy-simall/probBB
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       5   (56000,)   float64   


In [3]:
ffE.info()

Filename: cosmofast/planck_18/data/legacy-simall/probEE
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       5   (84000,)   float64   


In [4]:
testE = np.load("cosmofast/tests/test_simall_EE.npz")
testB = np.load("cosmofast/tests/test_simall_BB.npz")
testEB = np.load("cosmofast/tests/test_simall_EEBB.npz")
testE['test_param'].shape, testB['test_param'].shape, testEB['test_param'].shape

((31,), (31,), (61,))

In [5]:
from scipy.interpolate import CubicSpline
cc_ee = CubicSpline(np.arange(0, 0.0001*3000, 0.0001),
                    ffE[0].data.reshape((28, 3000)), axis=1)
c_ee = np.swapaxes(cc_ee.c, 0, 2).copy()
cc_bb = CubicSpline(np.arange(0, 0.0001*2000, 0.0001),
                    ffB[0].data.reshape((28, 2000)), axis=1)
c_bb = np.swapaxes(cc_bb.c, 0, 2).copy()

In [6]:
test_e = np.empty(1)
cf.planck_18._simall._simall_f(
    testE['test_param'][2:30], testE['test_param'][30], test_e,
    c_ee, 0.0001, 3000)
test_e, testE['test_value']

(array([-198.02824807]), array([-197.99117722]))

In [7]:
sum(np.diag(cc_ee(testE['test_param'][2:30] / 2 / np.pi * np.arange(2, 30) *
                  np.arange(3, 31) / testE['test_param'][30]**2)))

-198.02824807360398

In [8]:
test_b = np.empty(1)
cf.planck_18._simall._simall_f(
    testB['test_param'][2:30], testB['test_param'][30], test_b,
    c_bb, 0.0001, 2000)
test_b, testB['test_value']

(array([-194.9979613]), array([-194.95735006]))

In [9]:
sum(np.diag(cc_bb(testB['test_param'][2:30] / 2 / np.pi * np.arange(2, 30) *
                  np.arange(3, 31) / testB['test_param'][30]**2)))

-194.99796129510952

In [10]:
test_e + test_b, testEB['test_value']

(array([-393.02620937]), array([-392.94852728]))

In [11]:
test_e = np.empty(1)
cf.planck_18._simall._simall_f(np.zeros(28), 1., test_e, c_ee, 0.0001, 3000)
test_e # original implementation: array([-205.90806093])

array([-205.90806093])

In [12]:
test_b = np.empty(1)
cf.planck_18._simall._simall_f(np.zeros(28), 1., test_b, c_bb, 0.0001, 3000)
test_b # original implementation: array([-194.73314137])

array([-194.73314137])

In [13]:
np.savez_compressed('cosmofast/planck_18/data/simall_ee.npz',
                    c=c_ee, step=0.0001, n_step=3000)
np.savez_compressed('cosmofast/planck_18/data/simall_bb.npz',
                    c=c_bb, step=0.0001, n_step=2000)

In [3]:
import cosmofast as cf
import numpy as np
f_ee = np.load('cosmofast/planck_18/data/simall_ee.npz')
f_bb = np.load('cosmofast/planck_18/data/simall_bb.npz')
test_ee = np.load("cosmofast/tests/test_simall_ee.npz")
test_bb = np.load("cosmofast/tests/test_simall_bb.npz")

In [4]:
test_e = np.empty(1)
cf.planck_18._simall._simall_f(
    test_ee['test_param'][2:30], test_ee['test_param'][30], test_e,
    f_ee['c'], f_ee['step'], f_ee['n_step'])
test_e, test_ee['test_value'] # array([-198.02824807])

(array([-198.02824807]), array([-197.99117722]))

In [5]:
test_b = np.empty(1)
cf.planck_18._simall._simall_f(
    test_bb['test_param'][2:30], test_bb['test_param'][30], test_b,
    f_bb['c'], f_bb['step'], f_bb['n_step'])
test_b, test_bb['test_value'] # array([-194.9979613])

(array([-194.9979613]), array([-194.95735006]))